# HW4



## Baseline Model
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, model_selection, preprocessing
import random

## Data Preprocessing
---
First, read in the data


In [2]:
# read in data
data = pd.read_csv('online_shoppers_intention.csv', sep=',', index_col=False)
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


We can see that the features "Month", "VisitorType", "Weekend", and "Revenue" are not numbers, to make the analysis easier, we turn them into number.

From the preview of the data, we can know "Weekend" and "Revenue" are Boolean value, so we just turn False into 0, and True into 1, also, we know that "Revenue" can be used as the label, so rename it as "Target".

In [3]:
# map the boolean value
def map_bool(x):
    if x == False:
        return 0
    else:
        return 1
    
data['Weekend']= data['Weekend'].apply(map_bool)
data['Target']= data['Revenue'].apply(map_bool)
data = data.drop(['Revenue'], axis=1)

In [4]:
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Target
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,1,0


Now, we have to check what values does "Month" and "VisitorType" has.

In [5]:
#check the values of Month and VisitorType
print("Month")
print(data['Month'].value_counts())
print("\n")
print("VisitorType")
print(data['VisitorType'].value_counts())

Month
May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: Month, dtype: int64


VisitorType
Returning_Visitor    10551
New_Visitor           1694
Other                   85
Name: VisitorType, dtype: int64


We turn "Feb" to 2, "Mar" to 3, and so on in the feature "Month", and turn "Returning_Visitor" to 1, "New_Visitor" to 2, "Other" to 3 in the feature "VisitorType"

In [6]:
# map the features
def map_month(x):
    num = {
        "Jan": 1,
        "Feb": 2,   
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "June":6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }
    return num.get(x,0)

def map_visitor(x):
    if x == "Returning_Visitor":
        return 1
    elif x == "New_Visitor":
        return 2
    elif x == "Other":
        return 3
    else:
        return 0

data['Month']= data['Month'].apply(map_month)
data['VisitorType']= data['VisitorType'].apply(map_visitor)

In [7]:
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Target
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,1,1,0


Check the value again

In [8]:
#check the values of Month and VisitorType
print("Month")
print(data['Month'].value_counts())
print("\n")
print("VisitorType")
print(data['VisitorType'].value_counts())

Month
5     3364
11    2998
3     1907
12    1727
10     549
9      448
8      433
7      432
6      288
2      184
Name: Month, dtype: int64


VisitorType
1    10551
2     1694
3       85
Name: VisitorType, dtype: int64


In [9]:
# check if there is null value
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
Administrative             12330 non-null int64
Administrative_Duration    12330 non-null float64
Informational              12330 non-null int64
Informational_Duration     12330 non-null float64
ProductRelated             12330 non-null int64
ProductRelated_Duration    12330 non-null float64
BounceRates                12330 non-null float64
ExitRates                  12330 non-null float64
PageValues                 12330 non-null float64
SpecialDay                 12330 non-null float64
Month                      12330 non-null int64
OperatingSystems           12330 non-null int64
Browser                    12330 non-null int64
Region                     12330 non-null int64
TrafficType                12330 non-null int64
VisitorType                12330 non-null int64
Weekend                    12330 non-null int64
Target                     12330 non-null int64
dtypes: float

We can see that there is no null value in the data.

Because the dataset didn't have the train set and test set, so we have to split it by ourselves

In [10]:
def split(data):
    # control randomization for reproducibility
    np.random.seed(42)
    random.seed(42)
    train, test = model_selection.train_test_split(data)
    x_train = train.loc[:, train.columns != 'Target']
    y_train = train['Target']
    x_test = test.loc[:, test.columns != 'Target']
    y_test = test['Target']
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = split(data)

## Model
---

- ### Logistic Regression
    - Train accuracy  : 0.8830972207202336
    - Test accuracy   : 0.8738241972105092

In [11]:
# Logistic Regression

def evaluate(model, x_train, y_train, x_test, y_test):
    train_preds = model.predict(x_train)
    print("# of train predict 0")
    print(list(train_preds).count(0))
    print("# of train predict 1")
    print(list(train_preds).count(1))
    test_preds = model.predict(x_test)
    print("# of test predict 0")
    print(list(test_preds).count(0))
    print("# of test predict 1")
    print(list(test_preds).count(1))
    train_acc = metrics.accuracy_score(y_train, train_preds)
    test_acc = metrics.accuracy_score(y_test, test_preds)
    print('Train accuracy: %s' % train_acc)
    print('Test accuracy: %s' % test_acc)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(x_train, y_train)
clf.fit(x_train, y_train)
evaluate(clf, x_train, y_train, x_test, y_test)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# of train predict 0
8549
# of train predict 1
698
# of test predict 0
2865
# of test predict 1
218
Train accuracy: 0.8830972207202336
Test accuracy: 0.8738241972105092


- ### SVM
    - Train accuracy  : 89.06 %
    - Test accuracy   : 87.58 %

In [12]:
# SVM
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

X_train_std = scaler.transform(x_train)
X_test_std = scaler.transform(x_test)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Custom kernel function
def my_kernel(X, Y):
    return np.dot(X, Y.T)
  
penalty = 0.05

svm = SVC(C=penalty, kernel="linear", probability=True)
svm.fit(X_train_std, y_train.values)

acc_train = accuracy_score(y_train.values, svm.predict(X_train_std)) * 100
acc_test = accuracy_score(y_test.values, svm.predict(X_test_std)) * 100
print("Penalty = %.2f, Train accuracy = %.2f %%, Test accuracy = %.2f %%" % (penalty, acc_train, acc_test))
 

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


Penalty = 0.05, Train accuracy = 89.06 %, Test accuracy = 87.58 %


## Improvement

Because that the data is very unbalance, about 84.5% (10,422) of the data is negative class sample, I think one way to increase the accuracy is to get more positive class sample

To simulate the result of getting more positive class sample, I reduce the negative class sample and make it as much as the positive one, hope that the result will be better

In [13]:
# get negative class only
neg_data = data[data['Target']==0]

neg_data['Target'].value_counts()

0    10422
Name: Target, dtype: int64

In [14]:
# shuffle the negative class data
from sklearn.utils import shuffle
neg_data = shuffle(neg_data)

neg_data = neg_data[:1908]
neg_data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Target
2678,1,5.000000,0,0.0,17,463.666667,0.000000,0.024444,0.0,0.0,5,2,2,7,4,1,1,0
5014,0,0.000000,0,0.0,15,575.666667,0.043333,0.044444,0.0,0.0,5,2,2,2,2,1,0,0
2661,0,0.000000,0,0.0,2,83.000000,0.000000,0.050000,0.0,0.6,5,1,2,2,4,2,0,0
1986,0,0.000000,1,0.0,46,4053.092208,0.014184,0.029684,0.0,0.0,3,1,1,1,2,1,0,0
6273,15,304.583333,0,0.0,33,761.223333,0.007619,0.020794,0.0,0.0,10,3,2,1,3,1,1,0


In [15]:
# concate the negative data with positive data
new_data = pd.concat([neg_data,data[data['Target']==1]])

new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3816 entries, 2678 to 12313
Data columns (total 18 columns):
Administrative             3816 non-null int64
Administrative_Duration    3816 non-null float64
Informational              3816 non-null int64
Informational_Duration     3816 non-null float64
ProductRelated             3816 non-null int64
ProductRelated_Duration    3816 non-null float64
BounceRates                3816 non-null float64
ExitRates                  3816 non-null float64
PageValues                 3816 non-null float64
SpecialDay                 3816 non-null float64
Month                      3816 non-null int64
OperatingSystems           3816 non-null int64
Browser                    3816 non-null int64
Region                     3816 non-null int64
TrafficType                3816 non-null int64
VisitorType                3816 non-null int64
Weekend                    3816 non-null int64
Target                     3816 non-null int64
dtypes: float64(7), int64(11)

In [16]:
# split the data into train set and test set
x_ntrain, y_ntrain, x_ntest, y_ntest = split(new_data)

In [17]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(x_ntrain, y_ntrain)
clf.fit(x_ntrain, y_ntrain)
evaluate(clf, x_ntrain, y_ntrain, x_ntest, y_ntest)

# of train predict 0
1660
# of train predict 1
1202
# of test predict 0
535
# of test predict 1
419
Train accuracy: 0.8183088749126485
Test accuracy: 0.8165618448637316


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
# SVM

scaler = StandardScaler()
scaler.fit(x_ntrain)

X_ntrain_std = scaler.transform(x_ntrain)
X_ntest_std = scaler.transform(x_ntest)
  
penalty = 0.05

svm = SVC(C=penalty, kernel="linear", probability=True)
svm.fit(X_ntrain_std, y_ntrain.values)

acc_train = accuracy_score(y_ntrain.values, svm.predict(X_ntrain_std)) * 100
acc_test = accuracy_score(y_ntest.values, svm.predict(X_ntest_std)) * 100
print("Penalty = %.2f, Train accuracy = %.2f %%, Test accuracy = %.2f %%" % (penalty, acc_train, acc_test))
 

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


Penalty = 0.05, Train accuracy = 81.73 %, Test accuracy = 82.39 %


## Result

- Logistic Regression
    - Train accuracy : 0.8183088749126485
    - Test accuracy  : 0.8165618448637316
- SVM
    - Train accuracy = 81.73 %
    - Test accuracy = 82.39 %

結果並沒有如預期的使準確度上升，反而還下降，我認為其中的原因是原來的資料太不平衡了，所以只要大部分的答案都答0準確度就會很高，而在改變輸入的資料後，可以看到預測值為0的比例下降了。我覺得只要資料量增加的話效果應該會如預期的一樣讓準確度提高